# Import

In [3]:
%pip install matplotlib
%pip install fuzzywuzzy
%pip install gensim
import json
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import re
import statistics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
print(matplotlib.__version__)
from fuzzywuzzy import fuzz


import numpy as np
import pandas as pd
from IPython.display import display
from functools import partial
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
3.7.1


/home/keos_angie/Insync/aaldana@keos.co/Google Drive/00__PC_AZUL/music/venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Data

In [4]:
df = pd.read_csv('Data_completa.csv')

In [5]:
cuadReal = []

for k, i in enumerate(df.energy):
  if i > 0.5 and df.valence.iloc[k] > 0.5:
    cuadReal.append("1")
  elif i > 0.5 and df.valence.iloc[k] <= 0.5:
    cuadReal.append("2")
  elif i <= 0.5 and df.valence.iloc[k] > 0.5:
    cuadReal.append("4")
  elif i <= 0.5 and df.valence.iloc[k] <= 0.5:
    cuadReal.append("3")
  else:
    cuadReal.append("0")
df["cuadReal"] = cuadReal

# Lexicones

Por supuesto, aquí te muestro a qué cuadrante del modelo de Russell pertenece cada emoción de la lista que proporcionaste:

Anger (Ira) - Cuadrante 2 (Alto en valencia negativa, alto en arousal)
Anticipation (Anticipación) - Cuadrante 3 (Alto en valencia positiva, alto en arousal)
Disgust (Asco) - Cuadrante 2 (Alto en valencia negativa, alto en arousal)
Fear (Miedo) - Cuadrante 2 (Alto en valencia negativa, alto en arousal)
Joy (Alegría) - Cuadrante 1 (Alto en valencia positiva, bajo en arousal)
Negative (Negativo) - Cuadrante 2 (Alto en valencia negativa, alto en arousal)
Positive (Positivo) - Cuadrante 1 (Alto en valencia positiva, bajo en arousal)
Sadness (Tristeza) - Cuadrante 4 (Bajo en valencia positiva, bajo en arousal)
Surprise (Sorpresa) - Cuadrante 2 (Alto en valencia negativa, alto en arousal)
Trust (Confianza) - Cuadrante 3 (Alto en valencia positiva, alto en arousal)
Espero que esta información te sea útil.

In [ ]:
#!wget "https://drive.google.com/uc?export=view&id=1GzMWfZH9CYWCgOySb-Gqzu3timHLqbL9" -O emolex.zip
#!unzip emolex.zip

In [9]:
emolex = pd.read_csv("/home/keos_angie/Música/music/NRC-Emotion-Lexicon/OneFilePerLanguage/Spanish-NRC-EmoLex.txt", sep="\t")
emolex.head()

,English Word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,Spanish Word
0,aback,0,0,0,0,0,0,0,0,0,0,detrás
1,abacus,0,0,0,0,0,0,0,0,0,1,ábaco
2,abandon,0,0,0,1,0,1,0,1,0,0,abandonar
3,abandoned,1,0,0,1,0,1,0,1,0,0,abandonado
4,abandonment,1,0,0,1,0,1,0,1,1,0,abandono


In [10]:
emotions = emolex.columns[1:-1]
emotions

Index(['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative',
       'positive', 'sadness', 'surprise', 'trust'],
      dtype='object')

In [11]:
vocab = {}
for emotion in emotions:
    vocab[emotion] = emolex.query(f"{emotion} == 1")["Spanish Word"].to_list()

In [12]:
def emotion_rep(text):
    tokens = str(text).split()
    counts = np.zeros(shape=(len(vocab), ))
    for token in tokens:
        for i, emotion in enumerate(emotions):
            if token in vocab[emotion]:
                counts[i] += 1
    return counts

In [14]:
df["lexicones"] = df.Lyrics.apply(emotion_rep)

In [15]:
def emociones(emociones):
    numeros = emociones
    tercer_vector = [emotions[i] for i in range(len(numeros)) if int(numeros[i]) > 0]

    return (tercer_vector)

In [16]:
df["emociones"] = df.lexicones.apply(emociones)

In [17]:
dict_palabras = ['2','3','2','2','1','2','1','4','2','3']

In [18]:
def emociones1(emociones):
    numeros = emociones
    tercer_vector = [dict_palabras[i] for i in range(len(numeros)) if int(numeros[i]) > 0]

    return (tercer_vector)
df["emocionesLetra"] = df.lexicones.apply(emociones1)

In [21]:
# Obtener el elemento más repetido
df['emocion_mas_comun'] = df['emocionesLetra'].map(lambda x: statistics.mode(x)[0] if x else 0)

In [22]:
df.head(3)

,Letra,Idioma,lyrics_state,hasLetra,Lyrics,Lyrics_proces,songId,track_uri,track_name,artist_uri,...,dominance_tags,mbid,spotify_id,genre,cuadrante,cuadReal,lexicones,emociones,emocionesLetra,emocion_mas_comun
0,Data/Letras/1-'Till-I-Collapse-Eminem.txt,en,complete,1.0,TranslationsEspañolTürkçePortuguêsItalianoहिन्...,cause feel tired left left feel weak feel weak...,till-i-collapse-eminem.txt,spotify:track:4xkOaSrkexMciUUogZKVTS,Till I Collapse,spotify:artist:7dGJo4pcD2V6oG8kP0tJRR,...,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap,3,2,"[0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 6.0, 0.0, 0.0, ...","[anticipation, negative, positive, trust]","[3, 2, 1, 3]",3
1,Data/Letras/2-St.-Anger-Metallica.txt,en,complete,1.0,St. Anger Lyrics[Verse]\nSaint Anger 'round my...,saint anger round neck saint anger round neck ...,st.-anger-metallica.txt,spotify:track:3fOc9x06lKJBhz435mInlH,St. Anger,spotify:artist:2ye2Wgw4gimLv2eAKyk1NB,...,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal,3,2,"[0.0, 6.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, ...","[anticipation, negative]","[3, 2]",3
2,Data/Letras/3-Speedin'-Rick-Ross.txt,en,complete,1.0,Speedin’ Lyrics[Intro: Rick Ross]\nLegendary\n...,legendary runners know trilla dollar count acc...,speedin-rick-ross.txt,spotify:track:3Y96xd4Ce0J47dcalLrEC8,Speedin',spotify:artist:1sBkRIssrMs1AbVkOJbc7a,...,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap,3,2,"[0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, ...","[anticipation, negative]","[3, 2]",3


# Data

In [6]:
df.to_csv('Data_enrriquecida.csv', header=True, index=False)